## 1. 패키지 로드

In [3]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

import pandas as pd

from tqdm import tqdm_notebook
import tqdm

2022-06-22 09:01:33.252594: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-22 09:01:47.873482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-22 09:01:47.878997: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-22 09:01:47.881594: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-22 09:01:47.884429: I tensorflow/core/

## 2. KoGPT2 로드

In [5]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'lm_head.weight', 'transformer.h.4.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expe

## 3. 학습 데이터 로드

In [11]:
### 학습 데이터 로드t
train_data = pd.read_csv("./train data/train_gpt2_title.csv")
train_data = train_data.drop('Unnamed: 0', axis=1)
train_data = train_data.rename({'title' : '제목'}, axis=1)
train_data.head(2)

,제목,감정
0,저만치 혼자서,슬픔
1,달러구트 꿈 백화점,행복


## 4. 토큰화

In [12]:
# params setting
batch_size = 32

In [13]:
# 데이터 로드 및 토큰화 함수
def get_emo_data():
    for question, answer in zip(train_data.감정.to_list(), train_data.제목.to_list()):
        bos_token = [tokenizer.bos_token_id]
        eos_token = [tokenizer.eos_token_id]
        sent = tokenizer.encode('<usr>' + question + '<sys>' + answer) 
        yield bos_token + sent + eos_token

In [14]:
# 모델이 필요로하는 DataSet 생성
dataset = tf.data.Dataset.from_generator(get_emo_data, output_types=tf.int32)

In [15]:
# 입력데이터의 크기가 가변 일때 같은 크기로 읽을 수 있게 변환
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

## 5. 모델 학습

In [16]:
# 1, 옵티마이저 정의
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

# 2. Step 정의
steps = len(train_data) // batch_size + 1

# 3. 에포치 설정(학습횟수)
EPOCHS = 5

In [17]:
# 4. 모델 학습
for epoch in range(EPOCHS):
    epoch_loss = 0

    for batch in tqdm.tqdm_notebook(dataset, total=steps):
        with tf.GradientTape() as tape:
            result = model(batch, labels=batch)
            loss = result[0]
            batch_loss = tf.reduce_mean(loss)

        grads = tape.gradient(batch_loss, model.trainable_variables)
        adam.apply_gradients(zip(grads, model.trainable_variables))
        epoch_loss += batch_loss / steps

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

/tmp/ipykernel_30460/3735721202.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(dataset, total=steps):


  0%|          | 0/138 [00:00<?, ?it/s]

[Epoch:    1] cost = 2.0790751


  0%|          | 0/138 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.33383918


...

[Epoch:    3] cost = 1.02393377


  0%|          | 0/138 [00:00<?, ?it/s]

[Epoch:    4] cost = 0.836244285


  0%|          | 0/138 [00:00<?, ?it/s]

[Epoch:    5] cost = 0.743950963


## 6. 전체 모델 저장

In [18]:
model.save_pretrained('./model/Gen_title_GPT2_model.h5')

## 7. 모델 불러오기 테스트

In [19]:
load_model = TFGPT2LMHeadModel.from_pretrained('./model/Gen_title_GPT2_model.h5')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./model/Gen_title_GPT2_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [20]:
def return_answer_by_chatbot(user_text):
    sent = '<usr>' + user_text + '<sys>'
    input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
    input_ids = tf.convert_to_tensor([input_ids])
    output = load_model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
    sentence = tokenizer.decode(output[0].numpy().tolist())
    chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
    return chatbot_response

In [21]:
return_answer_by_chatbot('슬픔')

'천검지애 2'